In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [3]:
Dseq=pd.read_csv('Dseq_AC.csv')
Dseq['log2FoldChange']=pd.to_numeric(Dseq.log2FoldChange, errors='coerce')
Dseq=Dseq[Dseq['log2FoldChange'] !=-10]
Dseq=Dseq[Dseq['log2FoldChange'] !=10]
Dseq=Dseq[~pd.isnull(Dseq['log2FoldChange'])]
Dseq=Dseq.reset_index(drop=True)
Dseq_all=pd.read_csv('Dseq_AC.csv')
Dseq_all['log2FoldChange']=pd.to_numeric(Dseq_all.log2FoldChange, errors='coerce')
Dseq_all=Dseq_all[Dseq_all.pval <= 0.01]
Dseq_all=Dseq_all[Dseq_all['log2FoldChange'] <=0]
Dseq_all=Dseq_all[Dseq_all['log2FoldChange'] !=-10]
Dseq_all=Dseq_all[~pd.isnull(Dseq_all['Human ENS'])]
Dseq_all=Dseq_all.reset_index()

In [194]:
# Coex=pd.read_csv('GTEx.coexpTable.0.25.Liver.txt',sep='\t')
# tipe='nafld'
df_final=pd.read_csv('filtered_HCC.csv',index_col=0)
tipe='hcc'

In [195]:
# chunksize = 10
# for chunk in pd.read_csv('newCoexpByQuant.0.25.LIHC.csv', chunksize=chunksize,index_col=0):
#     colnames=list(chunk.columns)
#     break
# temp=[i for i in range(len(chunk.columns)) if chunk.columns[i] in Dseq_all['Human ENS'].tolist()]
# temp.insert(0,0)
# df=pd.read_csv('newCoexpByQuant.0.25.LIHC.csv',usecols=temp,index_col=0)

In [193]:
# temp=[i for i in range(len(chunk.columns)) if chunk.columns[i] in Dseq_all['Human ENS'].tolist()]
# temp.insert(0,0)
# temp=Dseq_all['Human ENS'].tolist()
# df=pd.read_csv('newCoexpByQuant.0.25.LIHC.csv',usecols=temp)
# temp=Dseq['Human ENS'].tolist()
# df2=df[df['Unnamed: 0'].isin(temp)]

ValueError: Usecols do not match names.

In [187]:
top_num=10
dic_only_sig={}
not_found=0
for i in range(Dseq_all.shape[0]):
    temp3=[]
    temp4=[]
    temp5=[]
    if i % 100 == 0:
        print(i)
    temp_gene=Dseq_all['Human ENS'].ix[i]
    temp_exp=Dseq_all['log2FoldChange'].ix[i]
    temp_pval=Dseq_all['pval'].ix[i]
    try:
        df_temp=df_final[unicode(temp_gene)]
        temp=df_temp.abs()
        temp=temp.sort_values(ascending=False).index.tolist()[0:top_num+1]
    except:
        not_found+=1
#         dic_only_sig[i]=[[temp_pval],[temp_exp],[temp_gene],'nf']
        continue
    temp_val=list(np.sign(df_temp.ix[temp].tolist()))
    temp2=Dseq[Dseq['Human ENS'].isin(temp)]['Human ENS'].tolist()
    temp2.insert(0,temp_gene)
    for j in range(len(temp2)):
        k=temp2.index(temp2[j])
        if j > k:
            continue
        else:
            temp4.append(Dseq[Dseq['Human ENS'].isin([temp2[j]])]['log2FoldChange'].tolist()[0])
            temp5.append(temp2[j])
    for j in range(len(temp4)):
        if np.sign(temp4[j]) != temp_val[j]:
            temp3.append(1)
        else:
            temp3.append(0)
    if (sum(temp3)==top_num+1):
        dic_only_sig[i]=[temp_val,temp4,temp5]

0
100
200
300
400
500
600


In [ ]:
# top_num=5
# dic_only_sig={}
# for i in range(Dseq_all.shape[0]):
#     temp3=[]
#     temp4=[]
#     temp5=[]
#     if i % 100 == 0:
#         print(i)
#     temp_gene=Dseq_all['Human ENS'].ix[i]
#     temp_exp=Dseq_all['log2FoldChange'].ix[i] 
#     df_temp=df_final[temp_gene].sort_values(temp_gene,ascending=False)
#     temp2=Dseq[Dseq['Human ENS'].isin(temp)]['Human ENS'].tolist()
#     temp2.insert(0,temp_gene)
#     for j in range(len(temp2)):
#         k=temp2.index(temp2[j])
#         if j > k:
#             continue
#         else:
#             temp3.append(Dseq[Dseq['Human ENS'].isin([temp2[j]])]['pval'].tolist()[0])
#             temp4.append(Dseq[Dseq['Human ENS'].isin([temp2[j]])]['log2FoldChange'].tolist()[0])
#             temp5.append(temp2[j])
#     if len(temp3)>top_num+1:
#         temp3=temp3[0:top_num+1]
#         temp4=temp4[0:top_num+1]
#         temp5=temp5[0:top_num+1]
#     check=all(k <=0.01 for k in temp3)
#     if (check == True and len(temp3) > 0):
#         dic_only_sig[i]=[temp3,temp4,temp5]

In [183]:
for i in dic_only_sig.keys():
    temp=dic_only_sig[i]
    print(temp[2][0])

ENSG00000142185
ENSG00000077092
ENSG00000135046
ENSG00000116785
ENSG00000155629
ENSG00000080986
ENSG00000125538
ENSG00000116785
ENSG00000204287
ENSG00000116785


In [188]:
Dseq_all[Dseq_all['Human ENS'].isin(['ENSG00000116785'])]

,index,A-C,symbol,Human ENS,baseMean,baseMean A,baseMean C,log2FoldChange,pval,padj
141,354,ENSRNOG00000021575,AABR07021096.1,ENSG00000116785,890.986193,679.928008,1144.256015,-0.750956,0.000018,0.000911
151,379,ENSRNOG00000030715,Cfh,ENSG00000116785,5934.584557,4382.614928,7796.948113,-0.831118,0.000031,0.001426
381,866,ENSRNOG00000042369,RGD1564614,ENSG00000116785,1136.932871,945.026411,1367.220622,-0.532820,0.002109,0.042609


In [170]:
temp_gene='ENSG00000116785'
df_final[temp_gene].reindex(df_final[temp_gene].abs().sort_values(ascending=False,inplace=False).index).tolist()[0:11]

[1.0,
 0.380046928371,
 0.372031620329,
 0.37138018231700004,
 0.365454267377,
 0.362099505875,
 0.361138692734,
 0.36079206568899996,
 0.336732992055,
 0.335331177827,
 -0.335313220197]

In [137]:
# # save_obj(dic_only_sig,str(top_num)+tipe+'big')
# coba=0
# dic_rem={}
# for i in dic_only_sig.keys():
#     temp=dic_only_sig[i]
#     check=all(k <0 for k in temp[1])
#     if check:
#         dic_rem[i]=[i,Dseq_all['Human ENS'].ix[i],Dseq_all['symbol'].ix[i],sum(temp[1]),len(temp[1])]
# Dseq_all.ix[dic_rem.keys()].sort_values('pval').to_csv('top'+str(top_num)+tipe+'.csv')
# for i in Dseq_all['Human ENS'].ix[dic_rem.keys()].tolist():
#     print(i)

ENSG00000167779
ENSG00000142945
ENSG00000143196
ENSG00000106829
ENSG00000137804
ENSG00000183864
ENSG00000116690
ENSG00000173641
ENSG00000080986
ENSG00000126705
ENSG00000135046
ENSG00000073849
ENSG00000204287
ENSG00000127526
ENSG00000091317
ENSG00000198734
ENSG00000142185
ENSG00000075420
ENSG00000170412
ENSG00000100979
ENSG00000125538
ENSG00000185482
ENSG00000173559
ENSG00000185483
ENSG00000106100
ENSG00000077092
ENSG00000118729
ENSG00000157617
ENSG00000136167
ENSG00000117399
ENSG00000047457


In [198]:
Dseq

,A-C,symbol,Human ENS,baseMean,baseMean A,baseMean C,log2FoldChange,pval,padj
0,ENSRNOG00000033680,Cyp2b1,ENSG00000197408,283.989378,506.212004,17.322226,4.869046,1.800000e-86,3.150000e-82
1,ENSRNOG00000010753,Aig1,ENSG00000146416,217.750707,383.851184,18.430136,4.380408,8.610000e-82,7.540000e-78
2,ENSRNOG00000010134,Acot2,ENSG00000119673,1165.271876,2015.902424,144.515218,3.802132,5.230000e-81,3.050000e-77
3,ENSRNOG00000055909,Apoa4,ENSG00000110244,7232.091487,900.028096,14830.567560,-4.042460,4.010000e-78,1.760000e-74
4,ENSRNOG00000055221,Acot1,ENSG00000119673,1521.353323,2786.541046,3.128055,9.798994,2.830000e-69,9.920000e-66
5,ENSRNOG00000018145,Crat,ENSG00000095321,1112.306731,1959.675971,95.463642,4.359520,3.510000e-67,1.030000e-63
6,ENSRNOG00000015003,Pex11a,ENSG00000166821,1231.690433,2115.848431,170.700836,3.631694,5.400000e-61,1.350000e-57
7,ENSRNOG00000016219,Vnn1,ENSG00000112299,1091.080595,1911.800462,106.216756,4.169849,3.170000e-54,6.940000e-51
8,ENSRNOG00000009597,Cyp4a1,NaN,15360.631740,27150.488240,1212.803932,4.484560,8.690000e-54,1.690000e-50
9,ENSRNOG00000020775,Cyp2b2,ENSG00000197408,1080.718643,1844.237643,164.495842,3.486902,1.220000e-53,2.130000e-50


In [185]:
f = open('test.txt','w')
f.write('source'+'\t'+'target'+'\n')
res=[]
for i in dic_only_sig.keys():
    if i % 10 == 0:
        print(i)
    temp=dic_only_sig[i]
    temp=temp[2]
    temp1=temp[0]
    res.append(temp1)
    temp=[k for k in temp if k != temp1]
    for j in temp:
        f.write(temp1+'\t'+j+'\n')
f.close()


f = open('selected.txt','w')
f.write('Nodes'+'\t'+'Availability'+'\n')
for i in range(len(res)):
    f.write(res[i]+'\t'+str(i)+'\n')
f.close()

130
270
400
520
560
320
360
500
510


In [186]:
dic_only_sig

{4: [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [-1.387804935,
   -0.8907387720000001,
   -0.7796273220000001,
   -0.685246021,
   -0.77856342,
   -0.49027964],
  ['ENSG00000182326',
   'ENSG00000205403',
   'ENSG00000159403',
   'ENSG00000139178',
   'ENSG00000157131',
   'ENSG00000149131']],
 7: [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [-1.287575227,
   -0.528105219,
   -0.5852789629999999,
   -0.820383442,
   -0.80173124,
   -0.16252924900000001],
  ['ENSG00000143196',
   'ENSG00000144668',
   'ENSG00000165124',
   'ENSG00000112936',
   'ENSG00000011465',
   'ENSG00000140092']],
 23: [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [-0.879343525,
   -0.77856342,
   -0.316468074,
   -0.419897759,
   -0.140597154,
   -0.241921418],
  ['ENSG00000039537',
   'ENSG00000157131',
   'ENSG00000132837',
   'ENSG00000145476',
   'ENSG00000111713',
   'ENSG00000101981']],
 32: [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [-3.207054421,
   -0.503570061,
   -0.9174434859999999,
   -4.746431901,
   -0.8204994390000001,
   -0.437802168

In [ ]:
dic={}
dic_only_down={}
dic_only_up={}
dic_only_sig={}
sum_val=[]
sum_val2=[]
sum_val3=[]
top_num=5
for i in range(Dseq_all.shape[0]):
    if i % 100 == 0:
        print(i)
    temp_gene=Dseq_all['Human ENS'].ix[i]
    if pd.isnull(temp_gene):
        continue
    temp_exp=Dseq_all['log2FoldChange'].ix[i]
    temp=Coex[Coex.gene1.isin([temp_gene])].gene2.tolist()
    if len(temp)>top_num:
        temp=temp[0:top_num]
    temp=list(set(Dseq['Human ENS'].tolist()).intersection(temp))
    temp2=Dseq[Dseq['Human ENS'].isin(temp)]['log2FoldChange'].tolist()
    temp.insert(0,temp_gene)
    temp2.insert(0,temp_exp)
    temp2=[k for k in temp2 if k != -10]
    temp2=[k for k in temp2 if k != 10]
    temp2=map(float,temp2)
    check=all(k <=0 for k in temp2)
    temp_pval=Dseq[Dseq['Human ENS'].isin(temp)]['pval'].tolist()
    temp3=[temp_pval,temp2,temp]
    sum_val2.append([i,sum(temp2),len(temp2),sum(temp2)/len(temp2),min(temp2)])
    dic[i]=temp3
    
    if (check==True and len(temp2)>0) :
        sum_val.append([i,sum(temp2),len(temp2),sum(temp2)/len(temp2),min(temp2)])
        dic_only_down[i]=temp3

    check=all(k <=0.01 for k in temp_pval)
    if check==True:
        temp3=Dseq[Dseq['Human ENS'].isin(temp)]['log2FoldChange'].tolist()
        check2=all(k <=0 for k in temp3)
        temp4=[temp_pval,temp3,temp]
        if check2==True:
            sum_val3.append([i,sum(temp3),len(temp3),sum(temp3)/len(temp3),min(temp3)])
            dic_only_sig[i]=temp4

In [ ]:
sum_val2=sorted(sum_val2, key=lambda x: x[1])
sum_val2

In [ ]:
k=1 #total score
sum_val2=sorted(sum_val2, key=lambda x: x[k])
dic2={}
nodup=[]
res=[]
num_co=[]
f = open('selected.txt','w')
f.write('Nodes'+'\t'+'Availability'+'\n')
for i in sum_val2:
    if i[2]>0:
        f.write(Dseq_all['Human ENS'].ix[i[0]].upper()+'\t'+str(len(res))+'\n')
        res.append(Dseq_all['Human ENS'].ix[i[0]].upper())
        num_co.append(i[2])
f.close()

for i in res:
    temp=Coex[Coex.gene1.isin([i])]['gene2']
    if temp.shape[0] > top_num:
        temp=temp[0:top_num]
    temp=temp[temp.isin(res)].tolist()
    if temp == []:
        nodup.append(i)
    else:
        dic2[i]=temp

In [ ]:
dic[297]

In [ ]:
f = open('test.txt','w')
f.write('source'+'\t'+'target'+'\n')
temp1=res
top_num=10
for i in range(len(temp1)):
    if i % 10 == 0:
        print(i)
    temp=Coex[Coex.gene1.isin([temp1[i]])].gene2.tolist()
    if len(temp) > top_num:
        temp=temp[0:top_num]
    temp3=list(set(temp).intersection(set(Dseq['Human ENS'].tolist())))
    temp_val=Dseq[Dseq['Human ENS'].isin(temp3)]['log2FoldChange'].tolist()
    check=all(k <=0 for k in temp_val)
    if check == False:
        continue
#     print(temp1[i],sum(temp_val))
    for k in temp3:
        f.write(temp1[i]+'\t'+k+'\n')
f.close()

In [ ]:
f = open('test.txt','w')
f.write('source'+'\t'+'target'+'\n')
temp1=res
top_num=10
for i in range(len(temp1)):
    if i % 10 == 0:
        print(i)
    temp=Coex[Coex.gene1.isin([temp1[i]])].gene2.tolist()
    if len(temp) > top_num:
        temp=temp[0:top_num]
    temp3=list(set(temp).intersection(set(Dseq['Human ENS'].tolist())))
    temp_val=Dseq[Dseq['Human ENS'].isin(temp3)]['pval'].tolist()
    check=all(k <=0.01 for k in temp_val)
    if check == False:
        continue
#     print(temp1[i],sum(temp_val))
    for k in temp3:
        f.write(temp1[i]+'\t'+k+'\n')
f.close()

In [ ]:
gene1=pd.read_excel('Group1.xlsx',header=None)[0].tolist()
gene1=map(str,gene1)
Dseq[Dseq['Human ENS'].isin(gene1)]

In [ ]:
coba=0
for i in sum_val2:
    try:
        temp=dic_only_sig[i[0]]
    except:
        continue
    if len(temp[0])==1:
        coba+=1
        print(temp[2][0])
    else:
        if temp[2][0] in gene1:
            coba+=1
            print(temp[2][0])
    if coba==15:
        break

In [ ]:
coba=0
for i in sum_val2[0:60]:
    temp=dic[i[0]]
    if len(temp[0])==1:
        coba+=1
        print(len(temp[0]))
    else:
        if temp[2][0] in gene1:
            coba+=1
            if len(temp[1]) == 5:
                temp2=Coex[Coex.gene1.isin([temp[1][0]])].gene2.tolist()
                temp2.insert(0,temp[2][0])
                temp3=set(temp2).intersection(set(Dseq['Human ENS'].tolist()))
                print(len(temp3))
            else:
                print(len(temp[1]))
                           
    if coba==15:
        break